In [1]:
USER = "Nunzio"

In [2]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/Shareddrives/Project_ML_23/" + USER + "/machine-learning-project"
    sys.path.insert(0,BASE_PATH)
    N_JOBS = 1
    sys.path.insert(0,BASE_PATH + "/src/utils")
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'cup2023','ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'cup2023')
else :
    N_JOBS = -1
    TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [3]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + "/src/utils")
else:
    sys.path.append('../utils')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')
from keras import backend as K


from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2, set_random_state
from NN import NeuralNetwork
from joblib import dump

set_random_state(42)

In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') # casting
df_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [5]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target_x,target_y,target_z
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [6]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

# TODO split potenzialmente variabile?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [7]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping

In [8]:
def build_model(architecture, activation, learning_rate, momentum, nesterov, weight_decay, input_dimension,
                output_dimension, dropout_input_rate, dropout_hidden_rate):

    nn = NeuralNetwork(
        architecture=architecture,
        activation=activation,
        learning_rate=learning_rate,
        momentum=momentum,
        use_nesterov=nesterov,
        weight_decay=weight_decay,
        input_dimension=input_dimension,
        output_dimension=output_dimension,
        dropout_input_rate=dropout_input_rate,
        dropout_hidden_rate=dropout_hidden_rate
    )


    return nn.build_model()

In [9]:
def create_model_wrapper(patience, architecture, activation, learning_rate, momentum, nesterov, weight_decay, input_dimension, output_dimension, dropout_input_rate, dropout_hidden_rate, epochs, batch_size):  
    def build_fn():
        model = build_model(architecture=architecture, activation=activation, learning_rate=learning_rate, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay, input_dimension=input_dimension, output_dimension=output_dimension, dropout_input_rate=dropout_input_rate, dropout_hidden_rate=dropout_hidden_rate)
        return model
    callbacks = [EarlyStopping(monitor='loss', patience=patience)]
    return KerasRegressor(build_fn, epochs=epochs, batch_size=batch_size, verbose=0, callbacks = callbacks)


In [11]:


pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', create_model_wrapper)
])

In [12]:
param_grid = {
    'neuralnetwork__model__architecture': [
        (64, 64),  
        (128, 64),
        (128, 128),
        (256, 128),  
        (256, 256)
    ],
    'neuralnetwork__model__activation': ['relu'],  
    'neuralnetwork__model__learning_rate': [1e-4, 1e-3, 1e-2],  
    'neuralnetwork__model__momentum': [0.0, 0.5, 0.9, 1.0], 
    'neuralnetwork__model__nesterov': [True, False], 
    'neuralnetwork__model__weight_decay': [1e-5, 1e-4, 1e-3, 1e-2],  
    'neuralnetwork__model__input_dimension': [10],  
    'neuralnetwork__model__output_dimension': [3],  
    'neuralnetwork__model__dropout_input_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],  
    'neuralnetwork__model__dropout_hidden_rate': [
        [0.0, 0.0], 
        [0.2, 0.2],
        [0.3, 0.3],
        [0.3, 0.4],
        [0.4, 0.4],  
        [0.5, 0.5]   
    ],
    'neuralnetwork__patience': [5,8,9,10],
    'neuralnetwork__epochs': [400],
    'neuralnetwork__batch_size': [16, 32, 64]
   
}


In [13]:
from sklearn.metrics import make_scorer

mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer}

In [14]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    refit='MEE',
    n_jobs=1,
    verbose= 4
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 207360 candidates, totalling 1036800 fits


AttributeError: 'function' object has no attribute 'set_params'

In [30]:
best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))

Best parameters: {'neuralnetwork__model__activation': 'relu', 'neuralnetwork__model__architecture': (128, 128), 'neuralnetwork__model__dropout_hidden_rate': [0.2, 0.2], 'neuralnetwork__model__dropout_input_rate': 0.1, 'neuralnetwork__model__input_dimension': 10, 'neuralnetwork__model__learning_rate': 0.01, 'neuralnetwork__model__momentum': 0.7, 'neuralnetwork__model__nesterov': False, 'neuralnetwork__model__output_dimension': 3, 'neuralnetwork__model__weight_decay': 0.0001}
Best MEE score: 5.25845630047978
R2: 0.9642826189881155
RMSE: 3.6474890401355275


In [13]:
final_model = grid_search.best_estimator_
final_model

Pipeline(steps=[('scaler', RobustScaler()),
                ('neuralnetwork',
                 KerasRegressor(batch_size=32, epochs=100, model=<function build_model at 0x000002143C1329E0>, model__activation='relu', model__architecture=(128, 128), model__dropout_hidden_rate=[0.2, 0.2], model__dropout_input_rate=0.1, model__input_dimension=10, model__learning_rate=0.01, model__momentum=0.7, model__nesterov=False, model__output_dimension=3, model__weight_decay=0.0001, verbose=0))])

In [19]:
final_model.named_steps['neuralnetwork'].model

<function __main__.build_model(architecture, activation, learning_rate, momentum, nesterov, weight_decay, input_dimension, output_dimension, dropout_input_rate, dropout_hidden_rate)>

In [28]:
import tensorflow.keras.utils as keras_utils

nn_model = final_model.named_steps['neuralnetwork'].model_
keras_utils.plot_model(nn_model, show_shapes=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Test of the model

In [31]:
y_pred = final_model.predict(X_test)

mee = mean_euclidean_error(y_test, y_pred)
r2 = multidim_r2(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print('MEE:', mee)
print('R2:', r2)
print('RMSE:', rmse)

MEE: 7.084823386551169
R2: 0.9429525983065116
RMSE: 5.03487122327381


# Save model

In [32]:
model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)

INFO:tensorflow:Assets written to: C:\Users\N97CA~1.CAN\AppData\Local\Temp\tmp_72g2l20\assets


INFO:tensorflow:Assets written to: C:\Users\N97CA~1.CAN\AppData\Local\Temp\tmp_72g2l20\assets


['..\\..\\trained_models\\cup2023\\NN_model.joblib']